In [31]:
%run basics_sensorSignal.py
%matplotlib

Using matplotlib backend: MacOSX


#### TAN = c(NH$_3$ + c(NH$_4^+$)

sensor response is sigmoidal. It mainly depends on the response time 

In [ ]:
c_nh3, step = 1.5, 0.25
t = np.linspace(0, 250, num=int(100/step+1))
dfSig1, t1_10, t1_90 = _sensor_response(t=t, t_resp=1., conc=c_nh3, function='increase')
dfSig2, t2_10, t2_90 = _sensor_response(t=t, t_resp=50., conc=c_nh3, function='increase')
dfSig3, t3_10, t3_90 = _sensor_response(t=t, t_resp=50., conc=c_nh3, function='decline')

# .........................................................................
# plot sensor response
fig, ax = plt.subplots(figsize=(7,4))
ax.set_title('Sensor response', loc='left')

# sensor 1
ax.plot(t, dfSig1, 'k', label='fast response')
ax.axhline(dfSig1['signal'].max()*0.1, lw=0.5, ls=':', color='darkorange'), 
ax.axhline(dfSig1['signal'].max()*0.9, lw=0.5, ls=':', color='darkorange')

# sensor 2
ax.plot(t, dfSig2, '-.k', label='slow response')
ax.axhline(dfSig2['signal'].max()*0.1, lw=0.5, ls=':', color='forestgreen'), 
ax.axhline(dfSig2['signal'].max()*0.9, lw=0.5, ls=':', color='forestgreen')

# sensor 3
ax.plot(t, dfSig3, ':k', label='declining response')
ax.axhline(dfSig3['signal'].max()*0.9, lw=0.5, ls=':', color='forestgreen'), 
ax.axhline(dfSig3['signal'].max()*0.1, lw=0.5, ls=':', color='forestgreen')

ax.set_xlabel('Time / s'), ax.set_ylabel('Signal / V')
plt.xlim(-5, 155)
sns.despine()
plt.legend(loc='lower right', bbox_to_anchor=(1., 0.5))

sensor signal depends on concentration as well

In [ ]:
start_time, end_time = 0, 60
nsample, nper = 10*(end_time-start_time), 3
dfconc1, D1, P1, t1 = _target_concentration(c_nh3=c_nh3, tstart=start_time, tstop=end_time)
dfconc2, D2, P2, t2 = _target_concentration(N=nsample, nP=nper, c_nh3=c_nh3, 
                                            tstart=start_time, tstop=end_time)

# .................................
fs = 10
fig1, ax1 = plt.subplots(figsize=(7,3))
ax1.set_xlabel('Time / s', fontsize=fs)
ax1.set_ylabel('Analyte concentration', fontsize=fs)
ax1.plot(dfconc1, color='k', lw=1., marker='.', fillstyle='none', 
         label='N:{:.0f}, period: {:.0f}s, freq: 1'.format(end_time-start_time, P1))
ax1.plot(dfconc2, color='darkorange', marker='.', fillstyle='none', lw=0.75,
         label='N:{:.0f}, period: {:.0f}s, freq: {:.0f}'.format(nsample, P1, nper))
ax1.legend(loc=0, fontsize=fs*0.7), ax1.set_ylim(-0.05, 1.6)
plt.tight_layout(), sns.despine()
plt.show()

time to combine

signal ~ concentration(t) * response time * some other factors

In [40]:
%run basics_sensorSignal.py

In [49]:
# gloabl parameter
c_nh3 = 2.5                          # maximal ammonia concentration 
# concentration fluctuation
tstart, tend, step = 0, 60*3, 0.05   # sec; time range of fluctuation
nP = 3.5                             # number of cycles per period
# sensor settings
tstart_sens = 0                      # sec; start time range of sensor response            
t_resp = 10.                          # sensor response time
t_sample = 10.                       # sampling rate of the sensor; sec

# .............................................
# target concentration
dfconc, D, P, trange = _target_concentration(nP=nP, c_nh3=c_nh3, tstart=tstart, tstop=tend)
print('puls width: ', D)
# .............................................
# period (in s) when a measurement point is taken
# tsample_, srate = _sampling_time(trange=trange, Wpulse=D, tstart=tstart, tend=tend,
#                                 sampling_rate=t_sample)
# print('sensor sampling rate: ', srate)

# sensor response - how much time until we reach conc_target
k = (1/0.9**2-1)*t_resp**2
sens_time0 = np.linspace(tstart_sens, t_resp*2, num=int((t_resp*2-tstart_sens)/step+1))
print('sensor response time parameter: ', k)

# .............................................
# plotting 
par_sens = dict({'max conc': c_nh3, 'response time': t_resp, # 'samling rate': srate,
                'response para': k})
arg_fig = dict({'figsize': (6,3), 'fontsize': 10,
                'title': 'sensor response to fluctuating analyte concentration'})

puls width:  25.714285714285715
sensor response time parameter:  23.456790123456784


In [65]:
%run basics_sensorSignal.py
# samprate, tsampling = srate, tsample
figsize, fs = arg_fig['figsize'], arg_fig['fontsize']

# .....................................................................................
fig, ax = plt.subplots(figsize=figsize)
fig.canvas.set_window_title(arg_fig['title'])

ax.set_xlabel('Time / s', fontsize=fs), ax.set_ylabel('Analyte concentration', fontsize=fs)
ax.set_ylim(-0.05, par_sens['max conc']*1.19)

# target concentration
ax.plot(dfconc, ':k', label='target concentration')

# timestamps when conc is changing 
n, tsample, t = 0, list(), 0
while t < tend:
    t = tstart + n*D
    tsample.append(tstart + n*D)
    n +=1
print(tsample)
ddata, c_apparent = dict(), 0
for en, t in enumerate(tsample):
    # closest time step in concentration fluctuation
    tc = min(dfconc.index, key=lambda x: abs(x - t))
    if tc != dfconc.index[-1]:
        print(tc, dfconc.loc[tc, 'signal'])
        # find position in list for previous concentration
        pos = [en for en in range(len(dfconc.index)) if dfconc.index[en] == tc][0]
        tc_1, tc1 = dfconc.index[pos - 1], dfconc.index[pos + 1]
        # update actual sensor measurement time +t
        if en == 0:
            sens_time = np.linspace(0, D, num=int((D - tstart_sens) / step + 1))
        else:
            sens_time = np.linspace(sens_time[-1], tc_1 + D, num=int((D - tstart_sens) / step + 1))
        
        # indicates the concentration change
        ax.axvline(t, color='darkorange', lw=0.5)

        # sensor signal response
        df = sensor_response(t=tc, t_1=tc_1, dfconc=dfconc, c_nh3=par_sens['max conc'], 
                             sens_time0=sens_time0, k=par_sens['response para'], 
                             c_apparent=c_apparent)

        ddata[tc], c_apparent = df, df['signal'].to_numpy()[-1]
        ax.plot(sens_time, df,lw=1.25, color='teal', label='sensor response')


#for en, t in enumerate(tsampling):
#        # arrows indicating that the sensor starts to react to the surrounding
#        ax.arrow(t, par_sens['max conc']*1.15, 0.0, (par_sens['max conc']-par_sens['max conc']*1.15)/2.5, fc="k",
#                 ec="k", head_width=.8, head_length=0.1)

ax.legend(['target concentration', 'sensor response'], loc=0, fontsize=fs * 0.7)
plt.tight_layout(), sns.despine(), plt.grid(False)
plt.show()

[0.0, 25.714285714285715, 51.42857142857143, 77.14285714285714, 102.85714285714286, 128.57142857142858, 154.28571428571428, 180.0]
0.0 2.5
case8 - increase 2.5 2.5 0


ValueError: x and y must have same first dimension, but have shapes (515,) and (401, 1)

In [71]:
sens_time0

array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95,  1.  ,  1.05,  1.1 ,  1.15,  1.2 ,  1.25,  1.3 ,
        1.35,  1.4 ,  1.45,  1.5 ,  1.55,  1.6 ,  1.65,  1.7 ,  1.75,
        1.8 ,  1.85,  1.9 ,  1.95,  2.  ,  2.05,  2.1 ,  2.15,  2.2 ,
        2.25,  2.3 ,  2.35,  2.4 ,  2.45,  2.5 ,  2.55,  2.6 ,  2.65,
        2.7 ,  2.75,  2.8 ,  2.85,  2.9 ,  2.95,  3.  ,  3.05,  3.1 ,
        3.15,  3.2 ,  3.25,  3.3 ,  3.35,  3.4 ,  3.45,  3.5 ,  3.55,
        3.6 ,  3.65,  3.7 ,  3.75,  3.8 ,  3.85,  3.9 ,  3.95,  4.  ,
        4.05,  4.1 ,  4.15,  4.2 ,  4.25,  4.3 ,  4.35,  4.4 ,  4.45,
        4.5 ,  4.55,  4.6 ,  4.65,  4.7 ,  4.75,  4.8 ,  4.85,  4.9 ,
        4.95,  5.  ,  5.05,  5.1 ,  5.15,  5.2 ,  5.25,  5.3 ,  5.35,
        5.4 ,  5.45,  5.5 ,  5.55,  5.6 ,  5.65,  5.7 ,  5.75,  5.8 ,
        5.85,  5.9 ,  5.95,  6.  ,  6.05,  6.1 ,  6.15,  6.2 ,  6.25,
        6.3 ,  6.35,

In [70]:
sens_time

array([ 0.        ,  0.05002779,  0.10005559,  0.15008338,  0.20011117,
        0.25013897,  0.30016676,  0.35019455,  0.40022235,  0.45025014,
        0.50027793,  0.55030573,  0.60033352,  0.65036131,  0.70038911,
        0.7504169 ,  0.80044469,  0.85047248,  0.90050028,  0.95052807,
        1.00055586,  1.05058366,  1.10061145,  1.15063924,  1.20066704,
        1.25069483,  1.30072262,  1.35075042,  1.40077821,  1.450806  ,
        1.5008338 ,  1.55086159,  1.60088938,  1.65091718,  1.70094497,
        1.75097276,  1.80100056,  1.85102835,  1.90105614,  1.95108394,
        2.00111173,  2.05113952,  2.10116732,  2.15119511,  2.2012229 ,
        2.25125069,  2.30127849,  2.35130628,  2.40133407,  2.45136187,
        2.50138966,  2.55141745,  2.60144525,  2.65147304,  2.70150083,
        2.75152863,  2.80155642,  2.85158421,  2.90161201,  2.9516398 ,
        3.00166759,  3.05169539,  3.10172318,  3.15175097,  3.20177877,
        3.25180656,  3.30183435,  3.35186215,  3.40188994,  3.45

Backup

In [ ]:
# gloabl parameter
c_nh3 = 2.5                          # maximal ammonia concentration 
# concentration fluctuation
tstart, tend, step = 0, 60*3, 0.05   # sec; time range of fluctuation
nP = 3.5                             # number of cycles per period
# sensor settings
tstart_sens = 0                      # sec; start time range of sensor response            
t_resp = 50.                         # sensor response time
t_sample = 13.                      # sampling rate of the sensor; sec

# .............................................
# target concentration
dfconc, D, P, trange = _target_concentration(nP=nP, c_nh3=c_nh3, tstart=tstart, tstop=tend)
print('puls width: ', D)
# .............................................
# period (in s) when a measurement point is taken
tsample, srate = _sampling_time(trange=trange, Wpulse=D, tstart=tstart, tend=tend,
                                sampling_rate=t_sample)
print('sensor sampling rate: ', srate)
# sensor response
sens_time0 = np.linspace(tstart_sens, srate, num=int((srate-tstart_sens)/step+1))

# ........................................................................................
# plotting 
par_sens = dict({'max conc': c_nh3, 'response time': t_resp, 'samling rate': srate})
arg_fig = dict({'figsize': (6,3), 'fontsize': 10,
                'title': 'sensor response to fluctuating analyte concentration'})
# .................................................................................
fig, ax = plot_sensorresponse(dfconc=dfconc, sens_time0=sens_time0, tsampling=tsample, 
                              samprate=srate, tstart_sens=tstart_sens, step=step, nP=nP, 
                              par_sens=par_sens, arg_fig=arg_fig)